In [1]:
import re

# Fields we want to keep in the output
KEEP_FIELDS = {
    "CCSDS_CDM_VERS", "CREATION_DATE", "ORIGINATOR", "MESSAGE_ID",
    "TCA", "MISS_DISTANCE", "RELATIVE_SPEED",
    "OBJECT", "OBJECT_DESIGNATOR", "CATALOG_NAME", "OBJECT_NAME",
    "INTERNATIONAL_DESIGNATOR", "EPHEMERIS_NAME",
    "COVARIANCE_METHOD", "MANEUVERABLE", "REF_FRAME",
    "X", "Y", "Z", "X_DOT", "Y_DOT", "Z_DOT",
    "CR_R", "CT_R", "CT_T", "CN_R", "CN_T", "CN_N",
    "CRDOT_R", "CRDOT_T", "CRDOT_N", "CRDOT_RDOT",
    "CTDOT_R", "CTDOT_T", "CTDOT_N", "CTDOT_RDOT", "CTDOT_TDOT",
    "CNDOT_R", "CNDOT_T", "CNDOT_N", "CNDOT_RDOT", "CNDOT_TDOT", "CNDOT_NDOT"
}

def clean_line(line):
    """
    Normalize a single key/value line (KEY = VALUE).
    """
    line = line.strip()

    if "=" not in line:
        return None

    key, val = line.split("=", 1)
    key = key.strip()
    val = val.strip()

    if key not in KEEP_FIELDS:
        return None

    return f"{key} \t= {val}"


def clean_kvn_file(input_path, output_path):
    cleaned = []

    with open(input_path, "r") as f:
        for raw in f:
            line = raw.strip()

            # Skip comments and blank lines
            if line.startswith("COMMENT") or line == "":
                continue

            formatted = clean_line(line)
            if formatted:
                cleaned.append(formatted)

    # Write output
    with open(output_path, "w") as f:
        for line in cleaned:
            f.write(line + "\n")

    print(f"Saved cleaned file to: {output_path}")


# ---------------------------
# TEST: Clean one KVN file
# ---------------------------

input_file = "/content/CDM_5_10520_sms9_11282159.kvn"
output_file = "example_cleaned.kvn"

clean_kvn_file(input_file, output_file)


Saved cleaned file to: example_cleaned.kvn


In [2]:
import re
import math

# Fields we want to keep in the output
KEEP_FIELDS = {
    "CCSDS_CDM_VERS", "CREATION_DATE", "ORIGINATOR", "MESSAGE_ID",
    "TCA", "MISS_DISTANCE", "RELATIVE_SPEED",
    "OBJECT", "OBJECT_DESIGNATOR", "CATALOG_NAME", "OBJECT_NAME",
    "INTERNATIONAL_DESIGNATOR", "EPHEMERIS_NAME",
    "COVARIANCE_METHOD", "MANEUVERABLE", "REF_FRAME",
    "X", "Y", "Z", "X_DOT", "Y_DOT", "Z_DOT",
    "CR_R", "CT_R", "CT_T", "CN_R", "CN_T", "CN_N",
    "CRDOT_R", "CRDOT_T", "CRDOT_N", "CRDOT_RDOT",
    "CTDOT_R", "CTDOT_T", "CTDOT_N", "CTDOT_RDOT", "CTDOT_TDOT",
    "CNDOT_R", "CNDOT_T", "CNDOT_N", "CNDOT_RDOT", "CNDOT_TDOT", "CNDOT_NDOT"
}

def clean_line(key, val):
    """
    Apply custom modifications to key/value pairs.
    """

    # Force REF_FRAME → EME2000
    if key == "REF_FRAME":
        return f"{key} \t= EME2000"

    # Normalize MANEUVERABLE field
    if key == "MANEUVERABLE":
        if val is None or val.strip() == "" or val.lower() == "nan":
            val = "YES"
        elif val.upper() != "NO":
            val = "YES"
        return f"{key} \t= {val}"

    # Default return
    return f"{key} \t= {val}"


def clean_kvn_file(input_path, output_path):
    cleaned = []
    found_maneuverable = False  # Track if MANEUVERABLE exists

    with open(input_path, "r") as f:
        for raw in f:
            line = raw.strip()

            # Skip comments and blank lines
            if line.startswith("COMMENT") or line == "":
                continue

            if "=" not in line:
                continue

            key, val = line.split("=", 1)
            key = key.strip()
            val = val.strip()

            if key not in KEEP_FIELDS:
                continue

            # Track if maneuverable field was present
            if key == "MANEUVERABLE":
                found_maneuverable = True

            formatted = clean_line(key, val)
            cleaned.append(formatted)

    # If MANEUVERABLE field never appears → add it as YES
    if not found_maneuverable:
        cleaned.append("MANEUVERABLE \t= YES")

    # Write output
    with open(output_path, "w") as f:
        for line in cleaned:
            f.write(line + "\n")

    print(f"Saved cleaned file to: {output_path}")


# ---------------------------
# TEST: Clean one KVN file
# ---------------------------

input_file = "/content/CDM_5_10520_sms9_11282159.kvn"
output_file = "example_cleaned.kvn"

clean_kvn_file(input_file, output_file)


Saved cleaned file to: example_cleaned.kvn
